In [1]:
from datasets import load_dataset

masc_train = load_dataset("pain/MASC", split="train",streaming=True,trust_remote_code=True)
masc_test=load_dataset("pain/MASC", split="test",streaming=True,trust_remote_code=True)
masc_validation=load_dataset("pain/MASC", split="validation",streaming=True,trust_remote_code=True)

c:\Users\DELL\Desktop\Master\Deep_Learning_2MPRITEL\pfa\pfa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the module from C:\Users\DELL\.cache\huggingface\modules\datasets_modules\datasets\pain--MASC\0caca60a40eca3b488a275196d6e701c1b8d9b6ab33091169370e3fb0513c20b (last modified on Mon Feb 12 21:42:39 2024) since it couldn't be found locally at pain/MASC, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from C:\Users\DELL\.cache\huggingface\modules\datasets_modules\datasets\pain--MASC\0caca60a40eca3b488a275196d6e701c1b8d9b6ab33091169370e3fb0513c20b (last modified on Mon Feb 12 21:42:39 2024) since it couldn't be found locally at pain/MASC, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from C:\Users\DELL\.cac

In [2]:
COLUMNS_TO_KEEP = ["text", "audio"]
all_columns =masc_train.column_names
columns_to_remove = set(all_columns) - set(COLUMNS_TO_KEEP)

masc_train = masc_train.remove_columns(columns_to_remove)
masc_test = masc_test.remove_columns(columns_to_remove)
masc_validation = masc_validation.remove_columns(columns_to_remove)



In [3]:
# Define the preprocessing function
import librosa
import numpy as np
sample_rate=16000
n_mels=128
hop_length=512
n_fft=1024
def preprocess_fn(batch):
    # Load audio file
    audio, sr = librosa.load(batch["audio"]["file_path"], sr=sample_rate)
    
    # Convert audio to mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=n_mels, hop_length=hop_length, n_fft=n_fft)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)  # Convert to dB scale
    
    batch["audio"]=mel_spectrogram
    return batch

# Map the preprocessing function to the dataset

masc_train_preprocessed = masc_train.map(preprocess_fn)
masc_test_preprocessed= masc_test.map(preprocess_fn)
masc_validation_preprocessed = masc_validation.map(preprocess_fn)

In [1]:
import tensorflow as tf
batch_size=32

def generator():
    for i in range(0, len(masc_train_preprocessed), batch_size):
        batch = masc_train_preprocessed[i:i+batch_size]
        texts = [sample['text'] for sample in batch]
        audios = [sample['audio'] for sample in batch]
        yield {'text': texts, 'audio': audios}

# Create a TensorFlow dataset from the generator
x_train = tf.data.Dataset.from_generator(
    generator,
    output_signature={
        'text': tf.TensorSpec(shape=(batch_size, None), dtype=tf.string),  # Assuming variable text length
        'audio': tf.TensorSpec(shape=(batch_size, None, 80), dtype=tf.float32)  # Assuming audio features shape
    }
)

In [3]:
import tensorflow as tf
from tf_keras import layers
import tf_keras
class ConformerBlock(layers.Layer):
    def __init__(self, dim, num_heads, mlp_dim, kernel_size=32, dropout=0.1):
        super(ConformerBlock, self).__init__()
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=dim)
        self.conv1d_1 = layers.Conv1D(filters=mlp_dim, kernel_size=1, activation='gelu')
        self.conv1d_2 = layers.Conv1D(filters=dim, kernel_size=kernel_size, padding='same')
        self.layer_norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout = layers.Dropout(dropout)
        
    def call(self, inputs):
        x = inputs
        # Self-attention
        attn_output = self.attention(x, x)
        attn_output = self.dropout(attn_output)
        out1 = self.layer_norm1(x + attn_output)
        
        # Feed-forward network
        ff_output = self.conv1d_1(out1)
        ff_output = self.conv1d_2(ff_output)
        ff_output = self.dropout(ff_output)
        out2 = self.layer_norm2(out1 + ff_output)
        
        return out2

class FastConformer(tf_keras.Model):
    def __init__(self, input_dim, num_classes, num_blocks=6, head_size=128, hidden_size=512, dropout=0.1):
        super(FastConformer, self).__init__()
        
        # Input convolutional layer
        self.conv = layers.Conv1D(filters=hidden_size, kernel_size=3, padding='same')
        
        # Conformer blocks
        self.blocks = [ConformerBlock(dim=hidden_size, num_heads=head_size, mlp_dim=hidden_size*4, dropout=dropout) 
                       for _ in range(num_blocks)]
        
        # Output layer
        self.global_pooling = layers.GlobalAveragePooling1D()
        self.fc = layers.Dense(num_classes)
        
        # Dropout
        self.dropout = layers.Dropout(dropout)
        
    def call(self, inputs):
        # Input convolution
        x = self.conv(inputs)
        
        # Conformer blocks
        for block in self.blocks:
            x = block(x)
        
        # Global average pooling
        x = self.global_pooling(x)
        
        # Dropout
        x = self.dropout(x)
        
        # Output layer
        x = self.fc(x)
        
        return x


In [4]:
# Assuming `train_dataset` is an instance of IterableDataset with format "tensorflow"

# Define the FastConformer model
model = FastConformer(input_dim= 80,num_classes= 28,num_blocks=6, head_size=128, hidden_size=512, dropout=0.1)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, epochs=10, batch_size=32)




Epoch 1/10


ValueError: in user code:

    File "c:\Users\DELL\Desktop\Master\Deep_Learning_2MPRITEL\pfa\pfa\lib\site-packages\tf_keras\src\engine\training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\DELL\Desktop\Master\Deep_Learning_2MPRITEL\pfa\pfa\lib\site-packages\tf_keras\src\engine\training.py", line 1370, in run_step  *
        outputs = model.train_step(data)
    File "c:\Users\DELL\Desktop\Master\Deep_Learning_2MPRITEL\pfa\pfa\lib\site-packages\tf_keras\src\engine\training.py", line 1147, in train_step  *
        y_pred = self(x, training=True)
    File "c:\Users\DELL\Desktop\Master\Deep_Learning_2MPRITEL\pfa\pfa\lib\site-packages\tf_keras\src\engine\training.py", line 558, in error_handler  *
        return fn(*args, **kwargs)
    File "c:\Users\DELL\Desktop\Master\Deep_Learning_2MPRITEL\pfa\pfa\lib\site-packages\tf_keras\src\engine\training.py", line 588, in __call__  *
        return super().__call__(*args, **kwargs)
    File "c:

In [74]:
import torch.nn as nn
class DownsamplingModule(nn.Module):
    def __init__(self, input_size, output_size=256, kernel_size=9):
        super(DownsamplingModule, self).__init__()
        self.conv = nn.Conv1d(input_size, output_size, kernel_size=kernel_size, stride=8, padding=kernel_size//2)
        self.bn = nn.BatchNorm1d(output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

class DepthwiseSeparableConvolution(nn.Module):
    def __init__(self, input_size, output_size, kernel_size=3):
        super(DepthwiseSeparableConvolution, self).__init__()
        self.depthwise_conv = nn.Conv1d(input_size, input_size, kernel_size=kernel_size, groups=input_size, padding=kernel_size//2)
        self.pointwise_conv = nn.Conv1d(input_size, output_size, kernel_size=1)
        self.bn = nn.BatchNorm1d(output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.depthwise_conv(x)
        x = self.pointwise_conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x



In [76]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


X_train=torch.tensor(masc_train_preprocessed.remove_columns("text"))
y_train=torch.tensor(masc_train_preprocessed.remove_columns("audio"))
X_val=torch.tensor(masc_validation_preprocessed.remove_columns("text"))
y_val=torch.tensor(masc_validation_preprocessed.remove_columns("audio"))




# Create DataLoader objects for training and validation sets
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define the model
class ModifiedConformer(nn.Module):
    def __init__(self, input_size=80, hidden_size=256, kernel_size=9):
        super(ModifiedConformer, self).__init__()
        self.downsampling = DownsamplingModule(input_size)
        self.depthwise_conv1 = DepthwiseSeparableConvolution(hidden_size, hidden_size)
        self.depthwise_conv2 = DepthwiseSeparableConvolution(hidden_size, hidden_size)
        # Add more layers as needed

    def forward(self, x):
        x = self.downsampling(x)
        x = self.depthwise_conv1(x)
        x = self.depthwise_conv2(x)
        # Add more layers as needed
        return x

# Initialize the model
input_size = X_train.shape[1]  # Assuming X contains preprocessed audio features
model = ModifiedConformer(input_size=input_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    # Calculate validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
    
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss / len(train_dataset)}, Validation Loss: {val_loss / len(val_dataset)}")

# Optionally, save the trained model
torch.save(model.state_dict(), 'modified_conformer_model.pth')


RuntimeError: Could not infer dtype of IterableDataset

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader 
from sklearn.model_selection import ParameterGrid
from datasets.iterable_dataset import IterableDataset
from itertools import cycle


# Step 1: Define your RNN-T model
class RNNTransducer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RNNTransducer, self).__init__()
        self.rnn = nn.RNN(input_size=input_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        output, _ = self.rnn(x)
        output = self.fc(output)
        return output

#define the dataset
class DataStream1(IterableDataset):

    def __init__(self) -> None:
        super().__init__()
        self.size_input = 7
        self.size_output = 1

    def generate(self):
        while True:
            x = torch.rand(self.size_input)
            y = torch.rand(self.size_output)
            yield x, y

    def __iter__(self):
        return iter(self.generate())

# Step 2: Define hyperparameter grid for tuning
param_grid = {
    'lr': [0.001, 0.01, 0.1],
    'batch_size': [32, 64, 128],
    'hidden_dim': [64, 128, 256],
    'num_layers': [1, 2, 3],
    'dropout': [0.1, 0.2, 0.3],
    'optimizer': [optim.Adam, optim.SGD],
    'epochs': [10, 20, 30]
}

# Step 3: Create hyperparameter combinations
hyperparam_combinations = list(ParameterGrid(param_grid))

""" # Step 4: Load your dataset
transform = YourTransforms()  # Define your data transforms if necessary
dataset = YourDataset(root='path/to/your/dataset', transform=transform) """

# Step 5: Split dataset into train and validation sets
# Split dataset or use cross-validation for hyperparameter tuning

# Step 6: Iterate over hyperparameter combinations
for hyperparams in hyperparam_combinations:
    # Step 7: Create DataLoader for training set with current hyperparameters
    train_loader = DataLoader(dataset=masc_train_preprocessed, batch_size=hyperparams['batch_size'], shuffle=True)

    # Step 8: Define model with current hyperparameters
    model = RNNTransducer(input_dim=..., hidden_dim=hyperparams['hidden_dim'], output_dim=...)
    criterion = nn.CrossEntropyLoss()
    optimizer = hyperparams['optimizer'](model.parameters(), lr=hyperparams['lr'])

    # Step 9: Train model with current hyperparameters
    for epoch in range(hyperparams['epochs']):
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Step 10: Evaluate model with current hyperparameters on validation set
    # Evaluate model performance and select best hyperparameters based on validation results


TypeError: IterableDataset.__init__() missing 1 required positional argument: 'ex_iterable'